In [6]:
import SimpleITK as sitk
import myshow
import sys
import os
from myshow import myshow, myshow3d

In [7]:
directory = "CovidCTScan/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-440808-000001/02-09-2005-NA-THORAX PE-.3254/5.000000-ARTERIALVENOUS AXIAL THIN-.3255"
img = '/Users/syn/PycharmProjects/pythonProject7/CovidCTScan/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-440808-000001/02-09-2005-NA-THORAX PE-.3254/5.000000-ARTERIALVENOUS AXIAL THIN-.3255/1-069.dcm'
testimg = '/Users/syn/PycharmProjects/pythonProject7/1-008.nrrd'

In [8]:
reader = sitk.ImageFileReader()
reader.SetFileName(img)
image = reader.Execute()

size = image.GetSize()
print("Image size:", size[0], size[1], size[2])

Image size: 512 512 1


In [9]:
seg = image>200
myshow(sitk.LabelOverlay(image, seg), "Basic Thresholding")

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [10]:
otsu_filter = sitk.OtsuThresholdImageFilter()
otsu_filter.SetInsideValue(0)
otsu_filter.SetOutsideValue(1)
seg = otsu_filter.Execute(image)
myshow(sitk.LabelOverlay(image, seg), "Otsu Thresholding")

print(otsu_filter.GetThreshold() )
myshow(image)

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

-2976.0


interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [11]:
print("Reading Dicom directory:", directory)
reader = sitk.ImageSeriesReader()

dicom_names = reader.GetGDCMSeriesFileNames(directory)
reader.SetFileNames(dicom_names)

image = reader.Execute()

size = image.GetSize()
print("Image size:", size[0], size[1], size[2])

#if "SITK_NOSHOW" not in os.environ:
    #sitk.Show(image, "Dicom Series")

Reading Dicom directory: CovidCTScan/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-440808-000001/02-09-2005-NA-THORAX PE-.3254/5.000000-ARTERIALVENOUS AXIAL THIN-.3255
Image size: 512 512 127


In [17]:
#Fast marching segmentation, can barely see the lungs

In [18]:
seed = (132,142,96)
feature_img = sitk.GradientMagnitudeRecursiveGaussian(image, sigma=.5)
speed_img = sitk.BoundedReciprocal(feature_img) # This is parameter free unlike the Sigmoid
myshow(speed_img)

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

In [19]:
seed = (256,256,96)
feature_img = sitk.GradientMagnitudeRecursiveGaussian(image, sigma=.5)
speed_img = sitk.BoundedReciprocal(feature_img) # This is parameter free unlike the Sigmoid
myshow(speed_img)

interactive(children=(IntSlider(value=63, description='z', max=126), Output()), _dom_classes=('widget-interact…

In [23]:
#Watershed

In [25]:
reader = sitk.ImageFileReader()
reader.SetFileName(img)
img = reader.Execute()

size = image.GetSize()
print("Image size:", size[0], size[1], size[2])

Image size: 512 512 1


In [26]:
myshow(img)

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [27]:
sigma=img.GetSpacing()[0]
level=4

In [28]:
feature_img = sitk.GradientMagnitude(img)
myshow(feature_img, "Edge Features")

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [40]:
ws_img = sitk.MorphologicalWatershed(feature_img, level=0, markWatershedLine=True, fullyConnected=False)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [30]:

from ipywidgets import interact, interactive, FloatSlider

def callback(feature_img,*args, **kwargs):
    ws_img = sitk.MorphologicalWatershed(feature_img,*args, **kwargs)
    
    myshow(sitk.LabelToRGB(ws_img), "Watershed Segmentation")
    
interact(lambda **kwargs: callback(feature_img, **kwargs),
             markWatershedLine=True,
             fullyConnected=False,
             level=FloatSlider(min=0, max=255, step=0.1, value=4.0) )

interactive(children=(Checkbox(value=True, description='markWatershedLine'), Checkbox(value=False, description…

<function __main__.<lambda>(**kwargs)>

In [31]:
min_img = sitk.RegionalMinima(feature_img, backgroundValue=0, foregroundValue=1.0, fullyConnected=False, flatIsMinima=True)
marker_img = sitk.ConnectedComponent(min_img)
myshow(sitk.LabelToRGB(marker_img), "Too many local minima markers")

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…

In [41]:
from ipywidgets import interact, interactive, FloatSlider

def callback(feature_img,*args, **kwargs):
    ws_img = sitk.MorphologicalWatershed(feature_img,*args, **kwargs)
    
    myshow(sitk.LabelToRGB(ws_img), "Watershed Segmentation")
    
interact(lambda **kwargs: callback(feature_img, **kwargs),
             markWatershedLine=True,
             fullyConnected=False,
             level=FloatSlider(min=0, max=255, step=0.1, value=90.0) )

interactive(children=(Checkbox(value=True, description='markWatershedLine'), Checkbox(value=False, description…

<function __main__.<lambda>(**kwargs)>

In [43]:

from ipywidgets import interact, interactive, FloatSlider

def callback(feature_img,*args, **kwargs):
    ws_img = sitk.MorphologicalWatershed(feature_img,*args, **kwargs)
    
    myshow(sitk.LabelToRGB(ws_img), "Watershed Segmentation")
    
interact(lambda **kwargs: callback(feature_img, **kwargs),
             markWatershedLine=True,
             fullyConnected=False,
             level=FloatSlider(min=0, max=255, step=0.1, value=60.0) )

interactive(children=(Checkbox(value=True, description='markWatershedLine'), Checkbox(value=False, description…

<function __main__.<lambda>(**kwargs)>